In [2]:
import os
import graphlab
sf = graphlab.SFrame.read_csv('/Users/gaozheng/Desktop/test.csv', header=False)

2016-04-04 14:33:58,811 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1459794837.log


Finished parsing file /Users/gaozheng/Desktop/test.csv

Parsing completed. Parsed 4 lines in 0.026052 secs.

This non-commercial license of GraphLab Create is assigned to gao27@indiana.edu and will expire on January 17, 2017. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file /Users/gaozheng/Desktop/test.csv

Parsing completed. Parsed 4 lines in 0.059019 secs.


Inferred types from first line of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
sf


X1,X2
alainconnes alain connesis one of the leading ...,class1
americannationalstandardsinstitute the american ...,class2
alberteinstein near thebeginning of his career ...,class1
austriangerman as germanis a pluricentric ...,class2


<h1>generating features</h1>

In [6]:
bow = graphlab.text_analytics.count_words(sf['X1'])


In [8]:
sf['bow'] = bow


In [9]:
sf['tfidf'] = graphlab.text_analytics.tf_idf(sf['bow'])


In [10]:
docs = sf['bow'].dict_trim_by_values(2)


<h1>text cleaning</h1>

In [11]:
docs = docs.dict_trim_by_keys(graphlab.text_analytics.stopwords(), exclude=True)


In [12]:
docs[0]


{'academy': 5,
 'algebras': 2,
 'connes': 3,
 'differential': 2,
 'early': 2,
 'geometry': 2,
 'including': 2,
 'medal': 2,
 'operator': 2,
 'physics': 2,
 'sciences': 5,
 'theory': 2,
 'work': 2}

In [15]:
sf

X1,X2,bow,tfidf
alainconnes alain connesis one of the leading ...,class1,"{'and': 7, 'work': 2,'baumconnes': 1, 'gold': ...","{'and': 0.0, 'work':1.3862943611198906, ..."
americannationalstandardsinstitute the american ...,class2,"{'operations': 1,'office': 1, 'nsi': 1, ...","{'operations':1.3862943611198906, ..."
alberteinstein near thebeginning of his career ...,class1,"{'pardon': 1,'lagrangian': 1, 'four': ...","{'pardon':1.3862943611198906, ..."
austriangerman as germanis a pluricentric ...,class2,"{'limited': 1, 'gr': 1,'sollte': 1, 'sleep': 1, ...","{'limited':1.3862943611198906, ' ..."


In [16]:
train_data, test_data = sf.random_split(0.8)


<h1>use different classifiers</h1>

In [18]:
model = graphlab.svm_classifier.create(train_data, target='X2',
                                    features = ['tfidf'])

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'tfidf' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

SVM:

--------------------------------------------------------

Number of examples          : 3

...

| 4         | 19       | 0.500000  | 1.020163     | 1.000000          |

| 5         | 20       | 0.500000  | 1.020759     | 1.000000          |

| 6         | 22       | 1.000000  | 1.021795     | 1.000000          |

+-----------+----------+-----------+--------------+-------------------+

SUCCESS: Optimal solution found.

<h1>evaluation</h1>

In [19]:
results = model.evaluate(test_data)


In [21]:
results

{'accuracy': 0.0, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 1
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |    class1    |      class2     |   1   |
 +--------------+-----------------+-------+
 [1 rows x 3 columns], 'f1_score': 0.0, 'precision': 0.0, 'recall': None}